In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchaudio
import numpy as np
import scipy
import sklearn
import pandas as pd
import PIL
import matplotlib
import tokenizers
import datasets
import transformers
import matplotlib.pyplot as plt
from collections import Counter
import datetime

# Data exploration

Here we explore each feature column and comment on how we are going to preprocess each one. We have a total of 140700 samples. The last feature is the label depression which we will train on.

## Id column



In [5]:
# Load the dataset
data = pd.read_csv("./data/train.csv")

# Print examples of the 'id' column
print("Examples of 'id' column:")
print(data["id"].head())

print("")

print("We have " ,data["id"].count()," amount of samples")

Examples of 'id' column:
0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

We have  140700  amount of samples


We will drop the id column since there is no correlation between this and the label.

## Name column

In [6]:
# Print examples of the 'Name' column
print("Examples of 'Name' column:")
print(data["Name"].head())

print("")

print("Number of unique names:", data["Name"].nunique())

Examples of 'Name' column:
0    Aaradhya
1       Vivan
2      Yuvraj
3      Yuvraj
4        Rhea
Name: Name, dtype: object

Number of unique names: 422


We thought about dropping the name column since we thought that this may not have any correlation to the label. But we realised that there might be a correlation, so we decided to keep it. The correlation being that for example having a "unattractive" name can affect your life in a bad way. 


For pre-processing this we want to give the name a value of how unique it is.

## Gender

In [7]:
# Print examples of the 'Name' column
print("Examples of 'Name' column:")
print(data["Gender"].head())

print("")

print("Number of unique genders:", data["Gender"].nunique())

Examples of 'Name' column:
0    Female
1      Male
2      Male
3      Male
4    Female
Name: Gender, dtype: object

Number of unique genders: 2


We will just encode male to 1 and female to 0.

## Age

In [8]:
print("Examples of 'Age' column:")
print(data["Age"].head())

print("")

print("Number of unique ages:", data["Age"].nunique())

Examples of 'Age' column:
0    49.0
1    26.0
2    33.0
3    22.0
4    30.0
Name: Age, dtype: float64

Number of unique ages: 43


We will only change the number from float to integer.

## City

In [9]:
print("Examples of 'City' column:")
print(data["City"].head())

print("")

print("Number of unique cities:", data["Age"].nunique())

Examples of 'City' column:
0         Ludhiana
1         Varanasi
2    Visakhapatnam
3           Mumbai
4           Kanpur
Name: City, dtype: object

Number of unique cities: 43


In [26]:
# Count occurrences of each unique city
city_counts = data["City"].value_counts()

# Find first occurrence index for each unique city
first_occurrence = data.groupby("City").apply(lambda x: x.index[0])

# Sort cities by count in descending order
sorted_cities = city_counts.index  # Cities sorted by count (default sorting from most to least)

count = 0
# Print results
print("Unique cities sorted by count (most to least):")
for city in sorted_cities:
    if city_counts[city] > 7:
        count += 1
        print(f"{city}: First Index = {first_occurrence[city]}, Count = {city_counts[city]}")

print(f"Total unique cities: {count}")


Unique cities sorted by count (most to least):
Kalyan: First Index = 36, Count = 6591
Patna: First Index = 9, Count = 5924
Vasai-Virar: First Index = 49, Count = 5765
Kolkata: First Index = 28, Count = 5689
Ahmedabad: First Index = 5, Count = 5613
Meerut: First Index = 17, Count = 5528
Ludhiana: First Index = 0, Count = 5226
Pune: First Index = 13, Count = 5210
Rajkot: First Index = 10, Count = 5207
Visakhapatnam: First Index = 2, Count = 5176
Srinagar: First Index = 26, Count = 5074
Mumbai: First Index = 3, Count = 4966
Indore: First Index = 189, Count = 4872
Agra: First Index = 18, Count = 4684
Surat: First Index = 20, Count = 4636
Varanasi: First Index = 1, Count = 4606
Vadodara: First Index = 43, Count = 4568
Hyderabad: First Index = 23, Count = 4496
Kanpur: First Index = 4, Count = 4398
Jaipur: First Index = 12, Count = 4328
Thane: First Index = 6, Count = 4289
Lucknow: First Index = 16, Count = 4280
Nagpur: First Index = 37, Count = 4209
Bangalore: First Index = 8, Count = 4123
C

C:\Users\elias\AppData\Local\Temp\ipykernel_15692\2661373420.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_occurrence = data.groupby("City").apply(lambda x: x.index[0])


We thought about encoding each city as the city's population density, but we found this to be challenging. So we will as for now just one-hot encode this.


But before one-hot encoding we need to edit the samples where the city does not make any sense. We spotted some interesting city names; 3.0 and Researcher... and so on. We have decided that we will assign this a missing value category for the samples where the city is only mentioned once.

# Pre-processing

Dropping the Id column

In [11]:
def dropId(data):

    # Drop the 'id' column
    data = data.drop(columns=["id"])
    return data


Encoding the name column

In [12]:
def name_freq(data):
    # Replace each name with its frequency in the dataset
    data["Name"] = data["Name"].map(data["Name"].value_counts())
    return data

Encoding gender column

In [13]:
def gender_encode(data):
    data["Gender"] = data["Gender"].map({"Female": 0, "Male": 1 })
    return data

Age

In [14]:
def age_integer(data):
    data["Age"] = data["Age"].astype(int)
    return data

City

In [27]:
def city_one_hot(data):
    data["City"] = ...

Pre-processing function:

In [12]:
def pre_processing(data):

    data = dropId(data)
    data = name_freq(data)
    data = gender_encode(data)
    data = age_integer(data)
    return data


data = pre_processing(data)

print(data.head())

   Name  Gender  Age           City Working Professional or Student  \
0  2045       0   49       Ludhiana            Working Professional   
1   963       1   26       Varanasi            Working Professional   
2   730       1   33  Visakhapatnam                         Student   
3   730       1   22         Mumbai            Working Professional   
4   499       0   30         Kanpur            Working Professional   

         Profession  Academic Pressure  Work Pressure  CGPA  \
0              Chef                NaN            5.0   NaN   
1           Teacher                NaN            4.0   NaN   
2               NaN                5.0            NaN  8.97   
3           Teacher                NaN            5.0   NaN   
4  Business Analyst                NaN            1.0   NaN   

   Study Satisfaction  Job Satisfaction     Sleep Duration Dietary Habits  \
0                 NaN               2.0  More than 8 hours        Healthy   
1                 NaN               3.0 